In [ ]:
import os

import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../Share")
import config, utils, baseline, Processing_same_with_MATLAB


fs = round(10e6 / 2048)  # 4883 Hz
lower_cutoff = 100
upper_cutoff = 600
filter_b, filter_a = Processing_same_with_MATLAB.cheby2(4, 30, [lower_cutoff / (fs/2), upper_cutoff / (fs/2)], btype='bandpass')

feat_mean_1ch = np.array([0.1, 0.1, 2.5, 0.0, 11.0, 229.0, 13.8, -11.0, 9.0, 3.0, 1.5, 0.0, 0.0, 2.8])
feat_std_1ch = np.array([0.02, 0.05, 0.65, 0.02, 4.43, 303.9, 6.85, 12.18, 2.87, 0.87, 0.21, 0.04, 6.68, 1.12])
feat_mean = np.tile(feat_mean_1ch, (4, 1))
feat_std = np.tile(feat_std_1ch, (4, 1))

extractor = Processing_same_with_MATLAB.EMGFeatureExtractor(feat_mean, feat_std, filter_b, filter_a, Norm_bool=False)
trainer = baseline.TremorModelTrainer(config, subject="Hunmin")

# Found that there are normalization applied, so I've tested it without using an normalization

In [ ]:
data_files = config.dataset_sub_H
default_path = config.default_path_sub_H

baseline_K = [1, 6, 10, 14, 18, 22, 26, 30, 34, 38]
for K in baseline_K:
    X_train_all, y_train_all, X_test_all, y_test_all = [], [], [], []
    ACC_all = []

    for idx, session_info in enumerate(data_files):
    #for idx, session_info in enumerate(['Exp_2025-06-27-v1/E9AD0E7DCC2B/']):
        #print(f"Dataset {idx + 1}/{len(data_files)} - Session {session_info}\n{'='*40}")
        path = os.path.join(default_path, f'{session_info}raw/')
        features, class_labels = [], []
        for c_idx, c in enumerate(config.classes_5):
            raw_data = os.listdir(path+c)
            mat = scipy.io.loadmat(path+c+raw_data[0])
            extractor.buffer = mat['Data_ADC']
            class_labels.append(mat['Data_Cls'].reshape(-1))
            features_per_cls = extractor.extract_features()
            features_per_cls = np.transpose(features_per_cls, (2, 0, 1))  # shape: (1729, 4, 14)
            features.append(features_per_cls)
            #print(features_per_cls.shape, mat['Data_Cls'].reshape(-1).shape)

        #print(session_info)
        X = np.concatenate(features, axis=0)
        y = np.concatenate(class_labels, axis=0)

        if X.shape[0] != y.shape[-1]:
            print(f"Incorrect shape between features and Class: {X.shape} and {y.shape}, {session_info}")
            break

        if idx < K:
            X_train, y_train, X_test, y_test = utils.split_data(X, y, ratio=0.9)
            X_train_all.append(X_train)
            y_train_all.append(y_train)
            X_test_all.append(X_test)
            y_test_all.append(y_test)

            # Concatenate all so far
            X_train_stacked = np.concatenate(X_train_all, axis=0)
            y_train_stacked = np.concatenate(y_train_all, axis=0)
            X_test_stacked = np.concatenate(X_test_all, axis=0)
            y_test_stacked = np.concatenate(y_test_all, axis=0)
            acc=0
        elif idx == K:
            print(f"\t Training {K}: ", X_train_stacked.shape, y_train_stacked.shape)
            acc, pre_trained_CNN = trainer.train_multiple_dataset(X_train, y_train, X_test, y_test)
        else:
            #Test with current data
            X = np.expand_dims(X, axis=-1)
            acc = pre_trained_CNN.evaluate(X, y, verbose=0)[1]*100
            print(f"\t Accuracy on unseen dataset {idx+1}: {acc:.4f}%")
        ACC_all.append(acc)
    pd.DataFrame(ACC_all).to_csv(f'../Results/WithoutNorm/Baseline_results_train_with_{K}data_noNorm_H.csv', index=False)

In [ ]:
b_path = 'C:/Users/hml76/PycharmProjects/Tremor_project_local/NCI_mindforce'
baseline_K_char = ['1', '6', '10', '14', '18', '22', '26', '30', '34', '38']
baselines, baselines_N = [], []

for idx, K in enumerate(baseline_K_char):
    baselines.append(b_path + f'/github/Results/Baseline_results_train_with_{K}data_H.csv')
    baselines_N.append(b_path + f'/github/Results/WithoutNorm/Baseline_results_train_with_{K}data_noNorm_H.csv')

for idx, b in enumerate(baseline_K_char):
    ACC_with_norm = float(np.mean(pd.read_csv(baselines[idx])['Accuracy'][int(baseline_K[idx]):]))
    ACC_without_norm = float(np.mean(pd.read_csv(baselines_N[idx])[int(baseline_K[idx]):]))
    print(f"Baseline K-{baseline_K_char[idx]}: Norm : {ACC_with_norm:.2f}% / Without Norm: {ACC_without_norm:.2f}%")


# Normalization is better
- How to set the normalization value remains a question